## Baseline experiments eith ESN

In [8]:
from tabulate import tabulate
from reservoirpy.nodes import (Reservoir, IPReservoir, NVAR, LMS, RLS, Ridge)
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)

# Define basic parameters
units = 100
lr = 0.3
sr = 1.25
ridge = 1e-5
delay = 5
order = 2
strides = 1

# Datasets
datasets = {
    'lorenz': lorenz(n_timesteps=3000),
    'mackey_glass': mackey_glass(n_timesteps=3000),
    'narma': narma(n_timesteps=3000)
}

results = {}

# Evaluation
for dataset_name, dataset in datasets.items():
    if dataset_name == 'lorenz':
        output_dim = 3
    else:
        output_dim = 1

    # Models
    models = {
        'ESN': (Reservoir(units=units, lr=lr, sr=sr), Ridge(output_dim=output_dim, ridge=ridge)),
        'ESN-IP': (IPReservoir(units=units, lr=lr, sr=sr), Ridge(output_dim=output_dim, ridge=ridge)),
        'ESN-LMS': (Reservoir(units=units, lr=lr, sr=sr), LMS(output_dim=output_dim)),
        'ESN-RLS': (Reservoir(units=units, lr=lr, sr=sr), RLS(output_dim=output_dim)),
        'NVAR-ESN': (NVAR(delay=delay, order=order, strides=strides), Ridge(output_dim=output_dim, ridge=ridge))
    }

    X = dataset[:-1]
    y_true = dataset[1:]

    X_train = X[:2000]
    X_test = X[2000:]
    y_train = y_true[:2000]
    y_test = y_true[2000:]

    for model_name, (reservoir, readout) in models.items():
        esn = reservoir >> readout

        if isinstance(readout, (LMS, RLS)):
            esn.train(X_train, y_train)
        else:
            esn.fit(X_train, y_train)

        y_pred = esn.run(X_test)

        mse_val = mse(y_test, y_pred)
        rmse_val = rmse(y_test, y_pred)
        rsquare_val = rsquare(y_test, y_pred)
        nrmse_val = nrmse(y_test, y_pred)
        
        results[f"{dataset_name} - {model_name}"] = {
            "RMSE": rmse_val,
            "R^2": rsquare_val,
            "NRMSE": nrmse_val,
            "MSE": mse_val
        }

# Prepare data for the table
table_data = []
headers = ["Dataset - Model", "RMSE", "R^2", "NRMSE", "MSE"]

for result_key, result_values in results.items():
    row = [result_key]
    for metric in headers[1:]:
        row.append(result_values[metric])
    table_data.append(row)

# Print the table
print(tabulate(table_data, headers=headers))


Running Model-71: 2000it [00:00, 22699.21it/s]         
Running Model-71: 100%|██████████| 1/1 [00:00<00:00, 10.83it/s]


Fitting node Ridge-45...


Running Model-71: 999it [00:00, 23793.25it/s]          
Running Model-72: 100%|██████████| 1/1 [00:00<?, ?it/s]


Fitting node IPReservoir-15...


Running Model-72: 2000it [00:00, 22203.60it/s]         
Running Model-72: 100%|██████████| 1/1 [00:00<00:00, 10.57it/s]


...

Fitting node Ridge-52...


Running Model-82: 999it [00:00, 22849.08it/s]          
Running Model-83: 999it [00:00, 24701.32it/s]          
Running Model-84: 999it [00:00, 24637.41it/s]          
Running Model-85: 2000it [00:00, 29098.92it/s]         
Running Model-85: 100%|██████████| 1/1 [00:00<00:00, 13.94it/s]


Fitting node Ridge-53...


Running Model-85: 999it [00:00, 29499.09it/s]          

Dataset - Model                 RMSE          R^2        NRMSE            MSE
-----------------------  -----------  -----------  -----------  -------------
lorenz - ESN              3.08154     0.955544     0.0460173      9.49587
lorenz - ESN-IP           0.251773    0.999703     0.00375978     0.0633896
lorenz - ESN-LMS         15.4481      0.064812     0.230689     238.642
lorenz - ESN-RLS          1.46785     0.989905     0.0219197      2.15457
lorenz - NVAR-ESN         0.00717963  1            0.000107215    5.15471e-05
mackey_glass - ESN        0.00248862  0.999883     0.0027104      6.19321e-06
mackey_glass - ESN-IP     0.029596    0.983494     0.0322336      0.000875921
mackey_glass - ESN-LMS    0.908097    0.000839027  0.989028       0.824639
mackey_glass - ESN-RLS    0.0059568   0.999331     0.00648768     3.54835e-05
mackey_glass - NVAR-ESN   0.00118134  0.999974     0.00128662     1.39557e-06
narma - ESN               0.0832699   0.175889     0.176861       0.00693388
narma 